In [1]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download piyushagni5/berlin-database-of-emotional-speech-emodb
! unzip berlin-database-of-emotional-speech-emodb.zip

 71% 27.0M/38.0M [00:00<00:00, 50.7MB/s]
100% 38.0M/38.0M [00:00<00:00, 96.0MB/s]
Archive:  berlin-database-of-emotional-speech-emodb.zip
  inflating: wav/03a01Fa.wav         
  inflating: wav/03a01Nc.wav         
  inflating: wav/03a01Wa.wav         
  inflating: wav/03a02Fc.wav         
  inflating: wav/03a02Nc.wav         
  inflating: wav/03a02Ta.wav         
  inflating: wav/03a02Wb.wav         
  inflating: wav/03a02Wc.wav         
  inflating: wav/03a04Ad.wav         
  inflating: wav/03a04Fd.wav         
  inflating: wav/03a04Lc.wav         
  inflating: wav/03a04Nc.wav         
  inflating: wav/03a04Ta.wav         
  inflating: wav/03a04Wc.wav         
  inflating: wav/03a05Aa.wav         
  inflating: wav/03a05Fc.wav         
  inflating: wav/03a05Nd.wav         
  inflating: wav/03a05Tc.wav         
  inflating: wav/03a05Wa.wav         
  inflating: wav/03a05Wb.wav         
  inflating: wav/03a07Fa.wav         
  inflating: wav/03a07Fb.wav         
  inflating: wav/03a07La.w

In [12]:
import os
from pydub import AudioSegment

for j, fn in enumerate(os.listdir('wav')):
  audio = AudioSegment.from_wav('wav/'+fn)
  if len(audio)>5000:
    print(fn)
    print(len(audio))

14b02Tc.wav
6215
12b09Td.wav
5414
11b03Td.wav
5905
14b03Ed.wav
5050
09a05Ed.wav
5157
16a05Ea.wav
5489
12b02Ea.wav
5567
14b03Ta.wav
5497
16b09Eb.wav
5026
16b03Ea.wav
5964
03b02Tb.wav
5127
08b03Tc.wav
8978
08b02Tc.wav
7679
12b03Ta.wav
5996
16b03Ta.wav
5976
09b03Ed.wav
5065
11a05Td.wav
5673
14a05Tc.wav
5029
11b03Wb.wav
5214
16a05Tb.wav
6788
15b03Tc.wav
5341
03b03Tc.wav
5299
08b09Tb.wav
6751
12a05Ta.wav
5360
08a05Ta.wav
5621


In [ ]:
import matplotlib.pyplot as plt
from scipy import signal
from scipy.io import wavfile as wav
import numpy as np
from numpy.lib import stride_tricks

In [ ]:
import librosa
import librosa.display

# Reading the audio file and applying some transformations (trimming, padding...) to "clean" the sound file

def read_audio(conf, pathname, trim_long_data):
    y, sr = librosa.load(pathname, sr=conf.sampling_rate)
    # trim silence
    if 0 < len(y): # workaround: 0 length causes error
        y, _ = librosa.effects.trim(y) # trim, top_db=default(60)
    # make it unified length to conf.samples
    if len(y) > conf.samples: # long enough
        if trim_long_data:
            y = y[0:0+conf.samples]
    else: # pad blank
        padding = conf.samples - len(y)    # add padding at both ends
        offset = padding // 2
        y = np.pad(y, (offset, conf.samples - len(y) - offset), 'constant')
    return y

# Thanks to the librosa library, generating the mel-spectogram from the audio file

def audio_to_melspectrogram(conf, audio):
    spectrogram = librosa.feature.melspectrogram(audio, 
                                                 sr=conf.sampling_rate,
                                                 n_mels=conf.n_mels,
                                                 hop_length=conf.hop_length,
                                                 n_fft=conf.n_fft,
                                                 fmin=conf.fmin,
                                                 fmax=conf.fmax,
                                                 win_length = conf.win_length)
    spectrogram = librosa.power_to_db(spectrogram)
    spectrogram = spectrogram.astype(np.float32)
    return spectrogram

# Adding both previous function together

def read_as_melspectrogram(conf, pathname, trim_long_data, debug_display=False):
    x = read_audio(conf, pathname, trim_long_data)
    mels = audio_to_melspectrogram(conf, x)
    return mels

# A set of settings that you can adapt to fit your audio files (frequency, average duration, number of Fourier transforms...)

class conf:
    # Preprocessing settings
    sampling_rate = 16000
    duration = 1
    win_length = 1024
    hop_length = 160 # to make time steps 128
    fmin = 0
    fmax = sampling_rate // 2
    n_mels = 128
    n_fft = 4096
    samples = sampling_rate * duration

In [ ]:
def mono_to_color(X, mean=None, std=None, norm_max=None, norm_min=None, eps=1e-6):
    # Stack X as [X,X,X]
    X = np.stack([X, X, X], axis=-1)

    # Standardize
    mean = mean or X.mean()
    std = std or X.std()
    Xstd = (X - mean) / (std + eps)
    _min, _max = Xstd.min(), Xstd.max()
    norm_max = norm_max or _max
    norm_min = norm_min or _min
    if (_max - _min) > eps:
        # Scale to [0, 255]
        V = Xstd
        V[V < norm_min] = norm_min
        V[V > norm_max] = norm_max
        V = 255 * (V - norm_min) / (norm_max - norm_min)
        V = V.astype(np.uint8)
    else:
        # Just zero
        V = np.zeros_like(Xstd, dtype=np.uint8)
    return V

In [ ]:
! mkdir images
! mkdir data
% cd data
! mkdir A
! mkdir E
! mkdir F
! mkdir L
! mkdir W 
! mkdir T
! mkdir N
% cd ..
! mkdir wav1

/content/data
/content


In [3]:
! pip3 install pydub

In [ ]:
import os
from pydub import AudioSegment

su = 0
for j, fn in enumerate(os.listdir('wav')): 
  # Input audio file to be sliced
  audio = AudioSegment.from_wav('wav/'+fn)
  # Length of the audiofile in milliseconds
  n = len(audio)
  # Variable to count the number of sliced chunks
  counter = 1
  interval = 1 * 1000
  overlap = 0.1 * 1000
  start = 0
  end = 0
  flag = 0
    
  for i in range(0, 2 * n, interval):
      su = su+1
      if i == 0:
          start = 0
          end = interval
      else:
          start = end - overlap
          end = start + interval 

      if end >= n:
          end = n
          flag = 1
    
      chunk = audio[start:end]
      filename = 'wav1/'+fn[5]+'chunk'+str(counter)+fn
      chunk.export(filename, format ="wav")
      counter = counter + 1
print(su)

3245


In [ ]:
import gc 

def save_image_from_sound1(img_path,directory):
    filename = rename_file(img_path)
    x = read_as_melspectrogram(conf, img_path, trim_long_data=False, debug_display=True)
    #x_color = mono_to_color(x)
    
    #plt.imshow(x, interpolation='nearest')
    plt.savefig('data/'+directory+'/'+filename)
    plt.show()
    
    plt.close()
    del x
    gc.collect()

In [ ]:
import shutil, os

for i, fn in enumerate(os.listdir('wav1')):
  if fn[0]=='A':
    save_image_from_sound1('wav1/'+fn,'A')
  elif fn[0]=='E':
    save_image_from_sound1('wav1/'+fn,'E')
  elif fn[0]=='F':
    save_image_from_sound1('wav1/'+fn,'F')
  elif fn[0]=='L':
    save_image_from_sound1('wav1/'+fn,'L')
  elif fn[0]=='N':
    save_image_from_sound1('wav1/'+fn,'N')
  elif fn[0]=='T':
    save_image_from_sound1('wav1/'+fn,'T')
  elif fn[0]=='W':
    save_image_from_sound1('wav1/'+fn,'W')

In [ ]:
import gc

def save_image_from_sound(img_path):
    filename = rename_file(img_path)
    x = read_as_melspectrogram(conf, img_path, trim_long_data=False, debug_display=True)
    #x_color = mono_to_color(x)
    
    plt.imshow(x, interpolation='nearest')
    plt.savefig('image/' + filename)
    plt.show()
    
    plt.close()
    del x
    gc.collect()

def rename_file(img_name):
    img_name = img_name.split("/")[1]
    img_name = img_name[:-4]
    img_name += ".jpg"
    return img_name

In [ ]:
import os 

for i, fn in enumerate(os.listdir('wav')):
    path = 'wav/' + fn
    save_image_from_sound1(path)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])